We will start by testing the backend of Polars

In [1]:
from pathlib import Path
import sys

from backend.polars_backend import (
    top_active_dates,
    top_emojis,
    top_mentioned_users
)

In [2]:
data_path = Path("../data/farmers-protest-tweets-2021-2-4.json")

In [3]:
n_dates = 10

active_dates = top_active_dates(data_path, n=n_dates)

print(f"**Top {n_dates} dates with the most tweets and their most active user:**\n")
for idx, (date, user) in enumerate(active_dates, start=1):
    print(f"{idx:2d}. Date: {date}   —   User with the most tweets: {user}")

**Top 10 dates with the most tweets and their most active user:**

 1. Date: 2021-02-12   —   User with the most tweets: RanbirS00614606
 2. Date: 2021-02-13   —   User with the most tweets: MaanDee08215437
 3. Date: 2021-02-17   —   User with the most tweets: RaaJVinderkaur
 4. Date: 2021-02-16   —   User with the most tweets: jot__b
 5. Date: 2021-02-14   —   User with the most tweets: rebelpacifist
 6. Date: 2021-02-18   —   User with the most tweets: neetuanjle_nitu
 7. Date: 2021-02-15   —   User with the most tweets: jot__b
 8. Date: 2021-02-20   —   User with the most tweets: MangalJ23056160
 9. Date: 2021-02-23   —   User with the most tweets: Surrypuria
10. Date: 2021-02-19   —   User with the most tweets: Preetm91


In [4]:
n_emojis = 10

most_used_emojis = top_emojis(data_path, n=n_emojis)

print(f"\n**Top {n_emojis} most used emojis (emoji — frequency):**\n")
for idx, (emoji, frequency) in enumerate(most_used_emojis, start=1):
    print(f"{idx:2d}. {emoji}   —   {frequency}")


**Top 10 most used emojis (emoji — frequency):**

 1. 🙏   —   7286
 2. 😂   —   3072
 3. 🚜   —   2972
 4. ✊   —   2411
 5. 🌾   —   2363
 6. ❤   —   1779
 7. 🤣   —   1668
 8. 👇   —   1108
 9. 💚   —   1040
10. 💪   —   947


In [5]:
n_mentions = 10

most_mentioned_users = top_mentioned_users(data_path, n=n_mentions)

print(f"\n**Top {n_mentions} most mentioned users (user — frequency):**\n")
for idx, (user, count) in enumerate(most_mentioned_users, start=1):
    print(f"{idx:2d}. {user}   —   {count}")


**Top 10 most mentioned users (user — frequency):**

 1. narendramodi   —   2265
 2. Kisanektamorcha   —   1840
 3. RakeshTikaitBKU   —   1644
 4. PMOIndia   —   1427
 5. RahulGandhi   —   1146
 6. GretaThunberg   —   1048
 7. RaviSinghKA   —   1019
 8. rihanna   —   986
 9. UNHumanRights   —   962
10. meenaharris   —   926
